In [1]:
# imports
import os
import time
import numpy as np
import pandas as pd
import json
import spotipy
import yt_dlp

from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# data loading
data_json = pd.read_json('song-ids.json')
data_json = pd.DataFrame(data_json)
data_json = data_json.rename(columns={0: 'song_id'})
data_json

,song_id
0,0pn7SynQPZBgNMlqPitrR6
1,1JsLA3JWc4XE2StbJcRYG6
2,23ssBTLuRpiE8hlCTEZCIk
3,6BPJCWBiFHF7ItUQlyyP6x
4,6GTVtjlDMEtnM2paZZ6VNe
...,...
24994,6Z1fY3aEN6dWMbLFSNCDYV
24995,2ljc7AncHJ7lZ7uTxALWLZ
24996,0B6g379TMw0nYGXA7y5xHz
24997,0VhkA3NAwIFfVW4NFCbAOD


In [3]:
# create a copy of the original dataframe (for appending song names)
data_json_copy = data_json
data_json_copy

,song_id
0,0pn7SynQPZBgNMlqPitrR6
1,1JsLA3JWc4XE2StbJcRYG6
2,23ssBTLuRpiE8hlCTEZCIk
3,6BPJCWBiFHF7ItUQlyyP6x
4,6GTVtjlDMEtnM2paZZ6VNe
...,...
24994,6Z1fY3aEN6dWMbLFSNCDYV
24995,2ljc7AncHJ7lZ7uTxALWLZ
24996,0B6g379TMw0nYGXA7y5xHz
24997,0VhkA3NAwIFfVW4NFCbAOD


In [4]:
# api creds
client_id = ''
client_secret = ''

In [5]:
# initializing api client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
# song ids
song_ids = data_json['song_id'].tolist()
song_ids = song_ids[15000:20000]
print(song_ids)

['3rEKEnavc5p0z1vwT791CR', '38GQUMciTXeuxcC3np9zDG', '1pZwsIB2ogBnbjydoIHQnL', '5msFXipAG1CsK92OakSAbS', '15EjbRnJutwfDcO8EW1HDJ', '3X7Iy2X4NRbb8gbDYb2xVq', '57jOEZtoLQK4zF2x55bdkp', '6pgKF5vcnMeFw0K2aZ6Gzl', '0yL7K9MzC20uBesZTPY81r', '5jFlj0Gzkk7mxdi8e4IlXx', '1kWBBODTsb5f3AT2bnJnlb', '28JBD8p18xNuOfyV7Cotdn', '4Gao22JfFCy34ya6MPlDrJ', '4rBO0G8nMiakstdKOJHf8T', '4qmVnESehucC98CkW9BCka', '1QnrrR1SCtizJUsmk4ZlFY', '3rfHoTV9wC0PEp4M64tebl', '0RGuXQ8szdBaiilq8FqXpT', '2gABnPYOMsyxThmsOQ4uWK', '7MlgVzcBOIzmcWKs2J3joZ', '7jNrOOifm0eBq3Pcmp4HrR', '5e574bhjycX1eH2l4Auage', '1Z55zokh2QnB3FE5T2OdZQ', '43izJQz4iVAlyYRkg2QDzg', '4BnrGx9tWNF8aiXl1UhDBa', '1kTGQL3OzcrnxUvGIugFnl', '1dekgAFF9uTCqLsklDaCWb', '6XGN9afKuLwmXEipg53yqz', '00X2yv2vrtritPt2CZnUTZ', '1zzbCMYlZZ8CcPFFUV9Kbm', '4kGoQLmgPxVcDzhdeqvlOp', '5mTbpbdBqzrpLlKXvsoexK', '2dTVbfJnOAe7Nm4ahrcknv', '3bDScAlZuzhkqENyhguNKN', '4NsPgRYUdHu2Q5JRNgXYU5', '0BTaaKT4RMbs5M73tOHX5Y', '19QGIdhhRCB303eGVUQyZM', '3uKqnQ0dGBcpovf9lV2nqg', '3I0aQw4LIw

In [7]:
# create download directory
download_dir = f"song_ids_downloads"
os.makedirs(download_dir, exist_ok=True)

In [8]:
# yt track download options
ydl_opts = {
    'outtmpl': f"{download_dir}/%(title)s.%(ext)s",
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

In [ ]:
# getting track details for entire list and downloading from youtube
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    for song_id in song_ids:
        track_info = sp.track(song_id)
        title = track_info['name']
        artist = track_info['artists'][0]['name']
        
        # start indexing from 1
        print(f"Downloading {song_ids.index(song_id)} of {len(song_ids)}")
        print(f"Downloading: {title} - {artist}")
        query = f"{title} {artist}"

        try:
            ydl.download([f"ytsearch:{query}"])
            info_query = ydl.extract_info(f"ytsearch:{query}", download=False)

'''
            # get url of the first search result
            url = info_query['entries'][0]['webpage_url']

            # get title of the downloaded video and store in video_title variable
            info_url = ydl.extract_info(url, download=False)
            video_title = info_url.get('title')
'''

            print(f"Downloaded: {title} - {artist}")

            # save title to a new column in the dataframe
            data_json_copy.at[song_ids.index(song_id)+15000, 'song_name'] = f"{song_id}.mp3"

        except yt_dlp.utils.DownloadError as e:
            print(f"Error downloading {title} - {artist}: {e}")
            data_json_copy.at[song_ids.index(song_id)+15000, 'song_name'] = f"NaN"

        # 10s timeout to avoid getting rate limited
        time.sleep(10)

In [ ]:
# updated dataframe
data_json_copy

In [ ]:
# save the new dataframe as a csv
data_json_copy.to_csv('data_json_copy4.csv', index=False)